In [1]:
# Cloning the YOLOv5 code base
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Installation of required dependencies
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 17075 (delta 18), reused 5 (delta 5), pack-reused 17049 (from 3)
Receiving objects: 100% (17075/17075), 15.70 MiB | 27.71 MiB/s, done.
Resolving deltas: 100% (11719/11719), done.
/kaggle/working/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 15.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import os
import shutil

# Data Path Configuration
input_image_dir = '/kaggle/input/kitti-dataset/data_object_image_2/training/image_2'
input_label_dir = '/kaggle/input/kitti-dataset/data_object_label_2/training/label_2'  
output_image_dir = './data/kitti/images'
output_label_dir = './data/kitti/labels'

# Creating a Destination Directory
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Copying image files
for img_file in os.listdir(input_image_dir):
    if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Make sure it's an image file
        shutil.copy(os.path.join(input_image_dir, img_file), os.path.join(output_image_dir, img_file))

# Converting KITTI labels to YOLO format
def convert_kitti_to_yolo(input_label_dir, output_label_dir):
    """
    Convert KITTI format labels to YOLO format.
    """
    for label_file in os.listdir(input_label_dir):
        label_path = os.path.join(input_label_dir, label_file)
        
        # Handles only files, skips directories
        if os.path.isfile(label_path) and label_file.endswith('.txt'):
            with open(label_path, 'r') as file:
                lines = file.readlines()

            yolo_lines = []
            for line in lines:
                data = line.split()
                if data[0] in ['Car', 'Pedestrian']:  # Retain only the Car and Pedestrian categories
                    cls_id = 0 if data[0] == 'Car' else 1
                    x1, y1, x2, y2 = map(float, data[4:8])

                    # Conversion to YOLO format (centroid coordinates + aspect normalization)
                    img_width, img_height = 1242, 375
                    x_center = (x1 + x2) / 2 / img_width
                    y_center = (y1 + y2) / 2 / img_height
                    width = (x2 - x1) / img_width
                    height = (y2 - y1) / img_height

                    yolo_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

            # Save converted labels
            output_file_path = os.path.join(output_label_dir, label_file)
            with open(output_file_path, 'w') as yolo_file:
                yolo_file.write("\n".join(yolo_lines))

# Perform data conversion
convert_kitti_to_yolo(input_label_dir, output_label_dir)
print("KITTI data has been successfully converted to YOLO format!")


KITTI data has been successfully converted to YOLO format!


In [3]:
# Create the data.yaml configuration file
data_yaml = """
train: /kaggle/working/yolov5/data/kitti/images 
val: /kaggle/working/yolov5/data/kitti/images

nc: 2
names: ['Car', 'Pedestrian']
"""

with open('./data/kitti.yaml', 'w') as f:
    f.write(data_yaml)

print("The data.yaml configuration is complete!")

The data.yaml configuration is complete!


In [4]:
# Training the YOLOv5 model
!python train.py --img 640 --batch 16 --epochs 20 --data ./data/kitti.yaml --weights yolov5s.pt --name kitti_yolo



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=./data/kitti.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, e

In [6]:
# Step 1: Validating Model Performance
# Evaluate the model performance using val.py
!python val.py --data ./data/kitti.yaml \
               --weights runs/train/kitti_yolo/weights/best.pt \
               --task val

# Step 2: Perform Detection on the Validation Dataset
# Use detect.py to test the trained model on the validation images
!python detect.py --weights runs/train/kitti_yolo/weights/best.pt \
                  --source /kaggle/working/yolov5/data/kitti/images \
                  --conf-thres 0.4 \
                  --save-txt \
                  --save-conf

# Step 3: Presentation of Detection Results
# Import required libraries
import cv2
import matplotlib.pyplot as plt
import glob

# Load the detection result images
result_images = glob.glob('./runs/detect/exp/*.jpg')  # Adjust path to YOLOv5 output folder

# Check if detection results are available
if not result_images:
    print("No detection result images found. Check the detect.py output path.")
else:
    # Display the first 4 detection result images
    plt.figure(figsize=(10, 10))
    for idx, img_path in enumerate(result_images[:4]):  # Show first 4 images
        img = cv2.imread(img_path)[:, :, ::-1]  # Convert BGR to RGB
        plt.subplot(2, 2, idx+1)
        plt.imshow(img)
        plt.title(f"Detection Result {idx+1}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()


val: data=./data/kitti.yaml, weights=['runs/train/kitti_yolo/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /kaggle/working/yolov5/data/kitti/labels.cache... 7481 images, 0 b
                 Class     Images  Instances          P          R      mAP50   
                   all       7481      33229      0.913      0.805      0.896      0.611
                   Car       7481      28742      0.941      0.914      0.971      0.762
            Pedestrian       7481       4487      0.886      0.695      0.821       0.46
Speed: 0.1ms pre-process, 2.0ms 